In [1]:
%matplotlib inline
import numpy as np
import sklearn.preprocessing as prep
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

tf 实现的自编码器 
这里会使用到一种参数初始化方式Xavier Initialization
这里实现标准的均匀分布的Xavier初始化器(目的是为了初始化权重大小)

In [2]:
#fan_in 输入点的数量
#fan_out 输出点的数量
def xavier_init(fan_in, fan_out, constant = 1):
    low = -constant * np.sqrt(6.0 / (fan_in + fan_out))
    high = constant * np.sqrt(6.0 / (fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out),minval = low, maxval = high,dtype = tf.float32)

In [3]:
#n_input 输入变量数
#n_hidden 隐藏层节点数
#transfer_function 激活函数
#optimizer优化器
#scale高斯噪声系数
class AdditiveGaussianNoiseAutoencoder(object):
    def __init__(self, n_input, n_hidden, transfer_function = tf.nn.softplus, optimizer = tf.train.AdamOptimizer(),scale = 0.1):
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.transfer = transfer_function
        self.scale = tf.placeholder(tf.float32) #定义成一个placeholder 偏向
        self.training_scale = scale
        network_weights = self._initialize_weights()
        self.weights = network_weights

        # 定义模型，也就是输入层，隐含层，输出层以及之间的映射矩阵
        self.x = tf.placeholder(tf.float32, [None, self.n_input]) #举重
        self.hidden = self.transfer(
            tf.add(
                tf.matmul(
                    self.x + scale * tf.random_normal((n_input,)), #计算公式 transfer使用激活函数
                    self.weights['w1']
                ),#带入变量
                self.weights['b1']
            )
        )
        self.reconstruction = tf.add(tf.matmul(self.hidden, self.weights['w2']), self.weights['b2']) #输出层不使用激活函数

        # 定义损失函数，这里我们使用平方差，因为下面的激活函数选择的是恒等
        self.cost = 0.5 * tf.reduce_sum(tf.pow(tf.subtract(self.reconstruction, self.x), 2.0))
        self.optimizer = optimizer.minimize(self.cost)#优化器为求损失极小化

        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)
        
    def _initialize_weights(self):
        # 字典类型
        all_weights = dict() 
        # 输入层到隐含层矩阵
        all_weights['w1'] = tf.Variable(xavier_init(self.n_input, self.n_hidden))# 使用xavier初始化变量
        # 输入层到隐含层偏置向量
        all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden], dtype = tf.float32))
        # 隐含层到输出层矩阵，可以看出是w1的逆
        all_weights['w2'] = tf.Variable(tf.zeros([self.n_hidden, self.n_input], dtype = tf.float32))
        # 隐含层到输出层偏置向量
        all_weights['b2'] = tf.Variable(tf.zeros([self.n_input], dtype = tf.float32))
        return all_weights
    
    # partial_fit()使用batch进行训练的函数，训练时使用六cost和optimizer，feed_dict喂数据，包括输入数据和高斯噪声系数。
    def partial_fit(self, X):
        cost, opt = self.sess.run((self.cost, self.optimizer),feed_dict = {self.x: X,self.scale: self.training_scale})
        return cost
    
    #   calc_total_cost()为只求损失不进行损失极小化操作，只执行cost
    def calc_total_cost(self, X):
        return self.sess.run(self.cost, feed_dict = {self.x: X,self.scale: self.training_scale})
    
    #   transform()根据输入获取其隐含层数据。
    def transform(self, X):
        return self.sess.run(self.hidden, feed_dict = {self.x: X,self.scale: self.training_scale})
    
    #generate()根据隐含层数据获取输出层数据。
    def generate(self, hidden = None):
        if hidden is None:hidden = np.random.normal(size = self.weights["b1"])
        return self.sess.run(self.reconstruction, feed_dict = {self.hidden: hidden})
    
    #  reconstruct()根据输入层数据获取输出层数据，相当于transform()+generate()。
    def reconstruct(self, X):
        return self.sess.run(self.reconstruction, feed_dict = {self.x: X,self.scale: self.training_scale})
    
    def getWeights(self):
        return self.sess.run(self.weights['w1'])

    def getBiases(self):
        return self.sess.run(self.weights['b1'])

In [4]:
mnist = input_data.read_data_sets('MNIST_data', one_hot = True)

#   standard_scale()对输入的图片数据和测试数据进行标准化，也就是让图片的像素值映射到0-1空间，这个时候我们需要使用sklearn.preprocessing中的StandardScaler。
def standard_scale(X_train, X_test):
    preprocessor = prep.StandardScaler().fit(X_train)
    X_train = preprocessor.transform(X_train)
    X_test = preprocessor.transform(X_test)
    return X_train, X_test

#  get_random_block_from_data()随机获取若干图片，我们不是把MNIST中所有数据进行训练。
def get_random_block_from_data(data, batch_size):
    start_index = np.random.randint(0, len(data) - batch_size)
    return data[start_index:(start_index + batch_size)]    

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
X_train, X_test = standard_scale(mnist.train.images, mnist.test.images)

n_samples = int(mnist.train.num_examples)
training_epochs = 20
batch_size = 128
display_step = 1

autoencoder = AdditiveGaussianNoiseAutoencoder(n_input = 784,
                                               n_hidden = 200,
                                               transfer_function = tf.nn.softplus,
                                               optimizer = tf.train.AdamOptimizer(learning_rate = 0.001),
                                               scale = 0.01)

for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(n_samples / batch_size)
    # 循环载入训练数据
    for i in range(total_batch):
        batch_xs = get_random_block_from_data(X_train, batch_size)

        # 使用batch喂入训练数据
        cost = autoencoder.partial_fit(batch_xs)
        # Compute average loss
        avg_cost += cost / n_samples * batch_size

    # 展示损失
    if epoch % display_step == 0:
        print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))

print("Total cost: " + str(autoencoder.calc_total_cost(X_test)))

Epoch: 0001 cost= 19611.083736364
Epoch: 0002 cost= 12107.804885227
Epoch: 0003 cost= 10620.842317045
Epoch: 0004 cost= 10308.985145455
Epoch: 0005 cost= 9158.406544886
Epoch: 0006 cost= 9386.579552841
Epoch: 0007 cost= 8847.634278409
Epoch: 0008 cost= 8442.755709091
Epoch: 0009 cost= 8826.129182386
Epoch: 0010 cost= 8314.348363068
Epoch: 0011 cost= 8589.324483523
Epoch: 0012 cost= 8268.158692045
Epoch: 0013 cost= 8704.360668750
Epoch: 0014 cost= 8140.573098295
Epoch: 0015 cost= 7836.033520455
Epoch: 0016 cost= 8458.704494886
Epoch: 0017 cost= 8519.741572159
Epoch: 0018 cost= 8225.359324432
Epoch: 0019 cost= 8132.735385795
Epoch: 0020 cost= 7793.019378409
Total cost: 700430.0
